In [1]:
import numpy as np
import pandas as pd
import random
import datetime as dt
import matplotlib.pyplot as plt
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import StandardScaler

df_locations = pd.read_csv("kil_me.csv").drop(['Unnamed: 0', 'country code', 'place_name', 'state_name',
       'state_code', 'county_name', 'county_code', 'community_name',
       'community_code', 'accuracy'], axis = 1)
df_locations.columns = ['zip5', 'latitude', 'longitude']

df_demo = pd.read_csv("citizens-home-financing-challenge/zip9_demographics_coded_pv.csv", dtype={'zip5': str}).drop(['age', 'household_count', 'person_count'], axis = 1)
df_demo.zip5 = df_demo.zip5.astype('int64')

In [2]:
majority_missing = ['mortgage2_limit','mortgage2_balance','mortgage3_limit','mortgage3_balance',
                    'mortgage4_limit','mortgage4_balance','mortgage5_limit','mortgage5_balance',
                    'homeequity1_limit','homeequity1_balance','homeequity2_limit','homeequity2_balance',
                    'homeequity3_limit','homeequity3_balance','homeequity4_limit','homeequity4_balance',
                    'homeequity5_limit','homeequity5_balance','total_homeequity_limit',
                    'total_homeequity_balance','homeequity1_loan_to_value']

sep_df = [pd.read_csv("citizens-home-financing-challenge/zip9_coded_20190" + str(i) + "_pv.csv", dtype={'zip5': str}).drop(columns=majority_missing) for i in range(4, 10)]
demo_df = pd.read_csv("citizens-home-financing-challenge/zip9_demographics_coded_pv.csv", dtype={'zip5': str})

sep_demo_merge = demo_df.copy()

for df in range(len(sep_df)):
    sep_demo_merge = sep_demo_merge.merge(sep_df[df], how='inner', on='zip9_code', suffixes=('','_df' + str(df + 4)), validate='one_to_one')
    
sep_demo_merge = sep_demo_merge.drop('zip9_code', axis = 1)
sep_demo_merge.zip5 = sep_demo_merge.zip5.astype('int64')

sep_demo_merge

,zip5,age,household_count,person_count,homebuyers,first_homebuyers,zip5_df4,bankcard_limit,bankcard_balance,bankcard_trades,...,homeequity2_open_df9,homeequity3_open_df9,homeequity4_open_df9,homeequity5_open_df9,total_homeequity_trades_df9,autoloan_open_df9,studentloan_open_df9,bankcard_open_df9,homeequity_open_df9,mortgage_open_df9
0,1001,50.365501,6,10,0,0,01001,35293.400000,17330.900000,4.400000,...,0.0,0.0,0.0,0.0,0.400000,0.100000,0.000000,1.000000,0.400000,0.800000
1,1001,35.258247,5,7,0,0,01001,6583.333333,1220.666667,1.666667,...,0.0,0.0,0.0,0.0,0.000000,0.428571,0.000000,0.714286,0.000000,0.000000
2,1001,46.101577,10,26,0,0,01001,25049.000000,5596.500000,2.769231,...,0.0,0.0,0.0,0.0,0.153846,0.307692,0.230769,0.807692,0.153846,0.423077
3,1001,44.539054,13,19,0,0,01001,13845.529412,4309.764706,2.764706,...,0.0,0.0,0.0,0.0,0.210526,0.526316,0.315789,0.947368,0.210526,0.473684
4,1001,57.670359,9,17,0,0,01001,45233.750000,6486.312500,4.125000,...,0.0,0.0,0.0,0.0,0.176471,0.235294,0.117647,1.000000,0.176471,0.294118
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6009254,99901,36.966252,7,11,0,0,99901,10505.555556,7403.666667,1.555556,...,0.0,0.0,0.0,0.0,0.000000,0.272727,0.272727,0.545455,0.000000,0.090909
6009255,99901,38.985856,8,15,0,0,99901,8793.333333,3752.866667,1.466667,...,0.0,0.0,0.0,0.0,0.000000,0.266667,0.200000,0.733333,0.000000,0.266667
6009256,99901,44.010514,5,8,0,0,99901,20125.000000,6851.375000,2.500000,...,0.0,0.0,0.0,0.0,0.000000,0.375000,0.000000,1.000000,0.000000,0.625000
6009257,99901,43.279977,8,13,0,0,99901,31040.666667,10886.500000,3.000000,...,0.0,0.0,0.0,0.0,0.000000,0.538462,0.307692,0.846154,0.000000,0.461538


In [3]:
print(len(df_demo))
df_demo = df_demo.groupby('zip5').sum().drop(['zip9_code'], axis = 1).reset_index()
print(len(df_demo))

print(len(df_locations))
df_demo_locations = pd.merge(df_demo, df_locations)
print(len(df_demo_locations))

print(len(sep_demo_merge))
sep_demo_merge = sep_demo_merge.groupby('zip5').mean().reset_index()
print(len(sep_demo_merge))

df = pd.merge(df_demo_locations, sep_demo_merge, on = 'zip5').sort_values('zip5', ascending = True)
print(len(df))

6009259
26598
26598
26598
6009259
26598
26598


In [17]:
df.zip5.values not in zips_to_throw_out

C:\Users\Jake\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  """Entry point for launching an IPython kernel.


True

In [18]:
zillow = pd.read_csv('final_clean2.csv').drop('Unnamed: 0', axis = 1)

zips_to_throw_out = np.setdiff1d(df.zip5.values, zillow.RegionName.values)

df = df[~df.zip5.isin(zips_to_throw_out)]

zillow['zip5'] = zillow.RegionName
zillow = zillow.drop('RegionName', axis = 1)
len(zillow)

26243

In [19]:
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
    
df = df.drop(columns = ['homebuyers_y', 'first_homebuyers_y','zip5'])
df.iloc[:, 2:] = StandardScaler().fit_transform(X = df.iloc[:, 2:].values)
df = df.fillna(0)
df = df.drop('first_homebuyers_x', axis = 1)
df.homebuyers_x = pd.qcut(df.homebuyers_x, 10, duplicates = 'drop').astype('category').cat.codes

C:\Users\Jake\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [20]:
df.to_csv("training_data.csv")

In [24]:
locations = pd.read_csv("kil_me.csv").drop('Unnamed: 0', axis = 1)
locations = locations[~locations.postal_code.isin(zips_to_throw_out)]
locations

In [26]:
locations.to_csv("kil_me.csv")